In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import roman
import collections

# SET PLOTTING SETTINGS
SMALL_SIZE = 16
MEDIUM_SIZE = 20
BIGGER_SIZE = 24
BIGGEST_SIZE = 28

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title

In [ ]:
def is_within_window(loc, upstream):
    '''Returns true if the sequence loc falls within a window contained in upstream.'''
    
    start = loc['start']
    end = loc['end']

    for i,row in upstream.iterrows():
        # the end falls into the upstream ORF interval
        if end <= row['end'] and end >= row['start']:
            return True
        # the start falls into the upstream ORF interval
        elif start <= row['end'] and start >= row['start']:
            return True
        # the binding event encompasses the entire upstream ORF interval
        elif start <= row['start'] and end >= row['end']:
            return True 
        
    return False

# Data loading and processing
Load our Excel file with the ChIP-exo data, find all genes with SNRs and p-values and save them in dictionaries for each PDM and experiment. Also load Ostrow et al. peak events and make sure they fall in 500 bp windows upstream of ORFs they identified. 

In [ ]:
experiments = ['Fkh1_log','Fkh1_stat','Fkh2_log','Fkh2_stat']

df_complete = pd.read_pickle('../Data/df_ORF_condition_normalized')

targets = {}; MACE = {}; GEM = {}; maxPeak = {}
for exp in experiments:
    # Load the targets: surpassed threshold in 2 or 3 PDMs
    df_targets = pd.read_excel('../Tables/targets_'+exp.replace('_',' ')+'.xlsx', index_col=0)
    targets[exp] = df_targets.index.tolist()
    
    # GEM
    df = pd.read_csv('../Data/ORF/GEM_'+exp+'.bed',sep='\t')
    df.columns = ['Chr','start','end','Gene','Gene_std','Score','Motif']
    # only genes with a SNR 
    df = df[df['Score'] > 0] 
    df = df.set_index('Gene')
    df.Chr = df.Chr.map(lambda x: roman.fromRoman(x))
    df = df.astype({'start':int,'end':int})
    GEM[exp] = df
    del(df)
    
    # MACE 
    df = pd.read_csv('../Data/ORF/MACE_'+exp+'.bed',sep='\t')
    df.columns = ['Chr','start','end','Gene','Score']
    # only genes with a p-value
    df = df[df['Score'] > 0]
    df = df.set_index('Gene')
    df.Chr = df.Chr.map(lambda x: x.replace('chr',''))
    df.Chr = df.Chr.map(lambda x: roman.fromRoman(x))
    df = df.astype({'start':int,'end':int})
    MACE[exp] = df
    del(df)
    
    # maxPeak
    df = pd.read_csv('../Data/ORF/condition_normalized/maxPeak_AD_12_'+exp+'.bed',sep='\t')
    df.columns = ['Chr','start','end','Gene','Score']
    # get rid of NA's due to zeros
    df = df[df['Score'] > 0]
    df = df.set_index('Gene')
    df.Chr = df.Chr.map(lambda x: x.replace('chr',''))
    df.Chr = df.Chr.map(lambda x: roman.fromRoman(x))
    df = df.astype({'start':int,'end':int})
    maxPeak[exp] = df
    del(df)
    
df_ostrow = pd.read_csv('../Data/Ostrow_et_al_2014/Table_S2.csv')
df_ostrow.columns = ['Index','Chr','start','end','class']
df_ostrow = df_ostrow[['Chr','start','end','class']]

df_ostrow_all_Fkh1 = df_ostrow[df_ostrow['class'].isin(['Fkh1-only','Fkh1and2'])]
df_ostrow_all_Fkh2 = df_ostrow[df_ostrow['class'].isin(['Fkh2-only','Fkh1and2'])]

# identify those peaks that are within 500 bp windows upstream of ORF starts
df_upstream = pd.read_excel('../Data/Ostrow_et_al_2014/Table_S4.xlsx', header=None, 
                           names=['Chr','start','end','gene','gene_std','Class'])

df_ostrow['upstream_orf'] = df_ostrow.apply(lambda x: is_within_window(x, df_upstream[df_upstream.Chr == x['Chr']]),axis=1)
df_ostrow = df_ostrow[df_ostrow['upstream_orf']]

df_ostrow_Fkh1 = df_ostrow[df_ostrow['class'].isin(['Fkh1-only','Fkh1and2'])]
df_ostrow_Fkh2 = df_ostrow[df_ostrow['class'].isin(['Fkh2-only','Fkh1and2'])]
df_ostrow = {'Fkh1':df_ostrow_Fkh1,'Fkh2':df_ostrow_Fkh2} 

# Count how many of our peak events overlap with Ostrow et al. enriched regions
Loop over the experiments, loop over each of the target genes identified in ChIP-exo, search for a peak window in the Ostrow dataset on the same chromosome. 

In [ ]:
overlap_counts = {exp:{'Overlap':0,'No overlap':0} for exp in experiments}
overlap_counts_all = {exp:{'Overlap':0,'No overlap':0} for exp in experiments}

for exp in experiments:
    for target in targets[exp]:
        std_name = df_complete.loc[target]['Standard name']
        
        if df_complete.loc[target]['maxPeak_AD_12 '+exp.replace('_',' ')] > 1:
            target_row = maxPeak[exp].loc[target]
        elif df_complete.loc[target]['GEM '+exp.replace('_',' ')] > 1:
            target_row = GEM[exp].loc[target]
        
        # if there were two peak events for this gene take the most significant one
        if not isinstance(target_row,pd.Series):
            target_row = target_row[target_row['Score'] == target_row['Score'].max()].iloc[0]
            
        chromo = target_row.Chr
        
        # look at overlap upstream of ORFs only
        if 'Fkh1' in exp:
            overlap = is_within_window(target_row, df_ostrow_Fkh1[df_ostrow_Fkh1.Chr == chromo])
        else:
            overlap = is_within_window(target_row, df_ostrow_Fkh2[df_ostrow_Fkh2.Chr == chromo])
            
        if overlap:
            overlap_counts[exp]['Overlap'] += 1
        else:
            overlap_counts[exp]['No overlap'] += 1
                
        # look at overlap with any enriched region
        if 'Fkh1' in exp:
            overlap = is_within_window(target_row, df_ostrow_all_Fkh1[df_ostrow_all_Fkh1.Chr == chromo])
        else:
            overlap = is_within_window(target_row, df_ostrow_all_Fkh2[df_ostrow_all_Fkh2.Chr == chromo])
            
        if overlap:
            overlap_counts_all[exp]['Overlap'] += 1
        else:
            overlap_counts_all[exp]['No overlap'] += 1
    
print('Overlap with upstream of ORFs')
counts = pd.DataFrame(overlap_counts)
counts.at['Total'] = counts.sum()
counts = counts.astype(int)
counts.at['Fraction'] = counts.apply(lambda x: round(x.Overlap / x.Total, 2))
display(counts)

print('Overlap with any enriched region')
counts_all = pd.DataFrame(overlap_counts_all)
counts_all.at['Total'] = counts_all.sum()
counts_all = counts_all.astype(int)
counts_all.at['Fraction'] = counts_all.apply(lambda x: round(x.Overlap / x.Total,2))
display(counts_all)

# Count how many of Ostrow et al.'s enriched regions contain a significant peak event

In [ ]:
# generate dictionaries of events above the thresholds for each PDM
maxPeak_sign = maxPeak.copy()
GEM_sign = GEM.copy()
MACE_sign = MACE.copy()

for exp in experiments:
    maxPeak_sign[exp] = maxPeak_sign[exp][maxPeak_sign[exp]['Score'] >= 1]
    GEM_sign[exp] = GEM_sign[exp][GEM_sign[exp]['Score'] >= 1]
    MACE_sign[exp] = MACE_sign[exp][MACE_sign[exp]['Score'] <= 0.005]


results = {exp:0 for exp in experiments}
for exp in experiments:
    
    if 'Fkh1' in exp:
        relevant_ostrow_df = df_ostrow_Fkh1
    else: 
        relevant_ostrow_df = df_ostrow_Fkh2
        
    for i,row in relevant_ostrow_df.iterrows():
        chromo = row.Chr

        for PDM in [maxPeak_sign[exp],GEM_sign[exp],MACE_sign[exp]]:
            contains_peak = is_within_window(row, PDM[PDM.Chr == chromo])

            if contains_peak:
                results[exp] += 1
                break

# normalize the counts to fraction of the total number of enriched regions
for exp in experiments:
    if 'Fkh1' in exp:
        relevant_ostrow_df = df_ostrow_Fkh1
    else: 
        relevant_ostrow_df = df_ostrow_Fkh2
        
    results[exp] = round(results[exp] / len(relevant_ostrow_df),2)

# show results
pd.DataFrame(results,index=['# enriched ChIP-chip regions containing a ChIP-exo peak'])

# Target IDs
For each experiment (outer key) get locations (Values) on each chromosome (key) for all our targets

In [ ]:
chromosomes = list(range(1,17))
chromosomes.append(1000) # 1000 is for M, which is the Mitochondrial DNA
target_markers_x = {exp:{chromo:[] for chromo in chromosomes} for exp in experiments}
target_markers_y = {exp:{chromo:[] for chromo in chromosomes} for exp in experiments}
target_markers_ID = {exp:{chromo:[] for chromo in chromosomes} for exp in experiments}

for exp in experiments:
    for g in targets[exp]:
        # get scores for g in exp for each PDM
        maxPeak_score = df_complete.loc[g]['maxPeak_AD_12 '+exp.replace('_',' ')]
        GEM_score = df_complete.loc[g]['GEM '+exp.replace('_',' ')]
        GEM_target = GEM_score >= 1
        MACE_score = df_complete.loc[g]['MACE '+exp.replace('_',' ')]
        MACE_target = MACE_score <= 0.005
        
        # Identify y coordinate for target labels 
        # use the MACE score if not significant in GEM
        # or if GEM had a relatively low score compared to MACE
        if MACE_target and ((not GEM_target) or (GEM_score <= 4 and MACE_score <= 0.001)  or ((GEM_score <= 2 and MACE_score <= 0.003))):
            row = MACE[exp].loc[[g]]
            if len(row) > 1:
                row = row[row['Score'] == MACE_score]
            
            target_markers_x[exp][row.Chr[0]].append(row.end[0] + 1)
            target_markers_y[exp][row.Chr[0]].append(row.Score[0])
            target_markers_ID[exp][row.Chr[0]].append(df_complete.loc[g]['Standard name'])
        else: 
            row = GEM[exp].loc[[g]]
            if len(row) > 1:
                row = row[row['Score'] == GEM_score]
            
            # if maxPeak scored higher: replace
            if maxPeak_score > GEM_score:
                row.Score = maxPeak_score
            
            target_markers_x[exp][row.Chr[0]].append(row.end[0] + 1)
            target_markers_y[exp][row.Chr[0]].append(row.Score[0])
            target_markers_ID[exp][row.Chr[0]].append(df_complete.loc[g]['Standard name'])
            
# fix duplicates by adding their IDs and deleting the second and third etc. occurences
for exp in experiments:
    for chromo in range(1,17):        
        # find duplicate x coordinates
        duplicates = [item for item, count in collections.Counter(target_markers_x[exp][chromo]).items() if count > 1]

        if len(duplicates) > 0:
            # find duplicate indices
            dup_indx = [ [i for i,val in enumerate(target_markers_x[exp][chromo]) if val==dup] for dup in duplicates]
            for dupl in dup_indx:
                # add the gene names
                target_markers_ID[exp][chromo][dupl[0]] = ', '.join([target_markers_ID[exp][chromo][i] for i in dupl])
                                    
            # delete the second, third etc. x, y and ID
            # do this in reverse sorted order so that the indices stay the same
            to_delete = [dupl[1:] for dupl in dup_indx]
            to_delete = sorted([y for x in to_delete for y in x]) # flatten
            for idx in reversed(to_delete):
                del target_markers_x[exp][chromo][idx]
                del target_markers_y[exp][chromo][idx]
                del target_markers_ID[exp][chromo][idx]

# Get all chromosome lengths 

In [ ]:
# loop over Ostrow, GEM, MACE and maxPeak
chromosome_length = {c:0 for c in chromosomes}
for TF in ['Fkh1','Fkh2']:
    for chromo in chromosomes:
        for exp in ['log','stat']:
            for (j,df,col) in [(0,df_ostrow[TF],'gray'),(1,GEM[TF+'_'+exp],'seagreen'), 
                               (2,maxPeak[TF+'_'+exp],'red'), (3,MACE[TF+'_'+exp],'blue')]:

                # filter on current chromosome
                df_chr = df[df.Chr == chromo] 

                if len(df_chr) > 0:
                    chr_length = int(df_chr['end'].max())

                    # keep track of length so that we can set x-range to include all events across all PDMs
                    if chr_length > chromosome_length[chromo]:
                        chromosome_length[chromo] = chr_length

chromosome_length

# Generate plots
Loop over the TFs and chromosomes and plot all PDMs and the Ostrow et al. events.

In [ ]:
plt.figure()    
for TF in ['Fkh1','Fkh2']:
    for chromo in chromosomes:
        for exp in ['log','stat']:
            fig, ax1 = plt.subplots()
            ax2 = ax1.twinx()

            # loop over Ostrow, GEM, MACE and maxPeak
            for (PDM,df,col) in [('Ostrow',df_ostrow[TF],'gray'),('maxPeak',maxPeak[TF+'_'+exp],'red'),
                               ('GEM',GEM[TF+'_'+exp],'seagreen'),('MACE',MACE[TF+'_'+exp],'blue')]:

                # filter on current chromosome
                df_chr = df[df.Chr == chromo] 
                
                if len(df_chr) > 0:
                    # get x positions 
                    x = np.array(list(range(chromosome_length[chromo])))
                                     
                    # plot axvspan's
                    # Note that axvspan interprets the ymin and ymax in an interval [0,1], 1 being the top of the plot
                    for i,row in df_chr.iterrows():
                        if PDM == 'Ostrow':
                            # arbitrary value so that the Ostrow peaks reach the top of the plot
                            a = 0.25
                            ax1.axvspan(xmin=row['start'],xmax=row['end'],ymin=0,ymax=1,color=col,alpha=a)
                            
                        elif PDM == 'GEM': # GEM
                            if row['Score'] >= 1:
                                a = 0.8
                            else:
                                a = 0.5
                                
                            ymax = np.log2(GEM[TF+'_'+exp].Score.max()) # ~4  
                            ylength = ymax + 1 # we start at 2**(-1)
                            yloc = np.log2(row['Score']) # where the label is and where the bar ends
                            yratio = (yloc + 1)/ylength # in [0,1]
                            ax1.axvspan(xmin=row['start'],xmax=row['end'],ymin=0,ymax=yratio,color=col,alpha=a)
                        elif PDM == 'maxPeak': # MaxPeak
                            if row['Score'] >= 1:
                                a = 0.5
                            else:
                                a = 0.2
                            
                            ymax = np.log2(GEM[TF+'_'+exp].Score.max()) # ~4  
                            ylength = ymax + 1 # we start at 2**(-1)
                            yloc = np.log2(row['Score']) # where the label is and where the bar ends
                            yratio = (yloc + 1)/ylength # in [0,1]
                            ax1.axvspan(xmin=row['start'],xmax=row['end'],ymin=0,ymax=yratio,color=col,alpha=a)
                        else:
                            if row['Score'] <= 0.005:
                                a = 0.8
                            else:
                                a = 0.5
                            
                            ymax = -4 # [0.0001,0.01]
                            ylength = 2
                            yloc = np.log10(row['Score']) # where the label is and where the bar ends
                            yratio = -1*(yloc + 2)/ylength # in [0,1]
                            ax2.axvspan(xmin=row['start'],xmax=row['end'],ymin=0,ymax=yratio,color=col,alpha=a)
                            
            # design axes
            ax1.set_yscale('log',basey=2)
            ax1.set_ylabel(r'$\log_2$(SNR)')
            ax1.yaxis.set_label_position("left")
            # set y axis limit based on max GEM score
            ax1.set_ylim([0.5,1.1*GEM[TF+'_'+exp].Score.max()])
            ax1.set_yticks([2**(int(l)) for l in np.arange(-1,1+np.floor(np.log2(1.1*GEM[TF+'_'+exp].Score.max())),step=1)])
            ax1.set_yticklabels([str(int(l)) for l in np.arange(-1,1+np.floor(np.log2(1.1*GEM[TF+'_'+exp].Score.max())),step=1)])
            ax1.set_xlabel('base pair location')
            ax1.set_xlim([0,chromosome_length[chromo]])
            
            num_x_labels = int(round(chromosome_length[chromo]/10000))
            if num_x_labels < 30:
                ax1.set_xticks([i*10000 for i in range(int(round(chromosome_length[chromo]/10000)))])
            elif num_x_labels < 45:
                ax1.set_xticks([i*25000 for i in range(int(round(chromosome_length[chromo]/25000)))])
            else:
                ax1.set_xticks([i*50000 for i in range(int(round(chromosome_length[chromo]/50000)))])
                
            for tick in ax1.get_xticklabels():
                tick.set_rotation(45)
                
            ax2.invert_yaxis()
            ax2.set_yscale('log',basey=10)
            ax2.set_ylabel(r'$\log_{10}$(p-value)')
            ax2.set_ylim([0.01, 0.0001])
            ax2.set_yticks([0.01,0.001, 0.0001])
            ax2.set_yticklabels(['-2','-3','-4'])
            ax2.set_xlim([0,chromosome_length[chromo]])
            ax2.get_xaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ','))) # thousands with comma separator

            gray_patch = mpatches.Patch(color='gray', label='Ostrow et al. 2014',alpha=0.5)
            seagreen_patch = mpatches.Patch(color='seagreen', label='GEM',alpha=0.5)
            red_patch = mpatches.Patch(color='red', label=r'$\it{maxPeak}$',alpha=0.5)
            blue_patch = mpatches.Patch(color='blue', label='MACE',alpha=0.5)
            ax1.legend(handles=[gray_patch,seagreen_patch,red_patch],loc=2)
            ax2.legend(handles=[blue_patch],loc=1)

            
            # Annotate genes that we consider as targets
            # use the SNR value of 1 to see whether we used the GEM/MaxPeak or MACE score for the y value
            for g_idx in range(len(target_markers_ID[TF+'_'+exp][chromo])):
                if target_markers_y[TF+'_'+exp][chromo][g_idx] >= 1:
                    label = target_markers_ID[TF+'_'+exp][chromo][g_idx]
                    
                    ax1.text(target_markers_x[TF+'_'+exp][chromo][g_idx],
                         target_markers_y[TF+'_'+exp][chromo][g_idx],
                        target_markers_ID[TF+'_'+exp][chromo][g_idx],fontsize=16)
                else: 
                    label = target_markers_ID[TF+'_'+exp][chromo][g_idx]
                    
                    ax2.text(target_markers_x[TF+'_'+exp][chromo][g_idx],
                         target_markers_y[TF+'_'+exp][chromo][g_idx],
                        target_markers_ID[TF+'_'+exp][chromo][g_idx],fontsize=16)
                    
            # draw cutoff markers on the two y-axes
#             m.set_clip_on(False)
            ax1.plot(range(chromosome_length[chromo]),[1 for i in range(chromosome_length[chromo])],'--g')
            ax2.plot(range(chromosome_length[chromo]),[0.005 for i in range(chromosome_length[chromo])],'--b')
#             m.set_clip_on(False)
            
            plt.title('Chromosome ' + roman.toRoman(chromo))

            fig.set_size_inches(25,5)
            fig.savefig('../Figures/Ostrow_comparison/'+TF+'_'+exp+'_'+'chr'+str(chromo)+'.png',
                        dpi=300, bbox_inches='tight')
            plt.show()